# gcp bigquery

In [11]:
# https://googleapis.dev/python/bigquery/latest/usage/pandas.html

In [12]:
# Cloud console way to run SQL
# https://console.cloud.google.com/bigquery

In [57]:
import geopandas

In [55]:
from google.cloud import bigquery
client = bigquery.Client()#location="EU")
print(f"Client creating using default project: {client.project}")

Client creating using default project: pythonproject-330909


To explicitly specify a project when constructing the client, set the project parameter

In [20]:
# client = bigquery.Client(location="US", project = "your-project-id")

In [ ]:
# Login to the VM using a Service Account that is authorised to access bigquery as admin
# Note: json containing key is stored on VM
# Steps set out in the following: -
# https://jingsblogcom.wordpress.com/2018/11/27/connect-your-jupyter-notebook-to-the-bigquery/amp/

In [32]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/markeholland70/key/pythonproject-330909-d0fd17649eb3.json'

In [45]:
QUERY = """
SELECT * 
FROM pythonproject-330909.datascience.fraud_dataset_v1 
LIMIT 1000
"""

In [46]:
# Run the query and get the data
query_job = client.query(QUERY)

In [47]:
df = query_job.to_dataframe()
df.head()

,WAERS,BUKRS,KTOSL,BELNR,BSCHL,HKONT,DMBTR,WRBTR,label
0,N09,A43,U50,C72,I53,F05,9.244551e+07,5.958504e+07,global
1,T68,A89,L29,H65,I26,Q69,9.244550e+07,5.958502e+07,global
2,D88,B37,D51,F69,N50,M13,9.244552e+07,5.958504e+07,global
3,E26,B76,C39,U14,R07,T45,9.244552e+07,5.958504e+07,global
4,C1,C10,C1,C10,A1,B1,1.542210e+06,0.000000e+00,regular


In [48]:
sql = """
    SELECT name, SUM(number) as count
    FROM bigquery-public-data.usa_names.usa_1910_current
    GROUP BY name
    ORDER BY count DESC
    LIMIT 10
"""

df = client.query(sql).to_dataframe()

In [49]:
df

,name,count
0,James,5041499
1,John,4902713
2,Robert,4758640
3,Michael,4387274
4,William,3927657
5,Mary,3755452
6,David,3623464
7,Richard,2548809
8,Joseph,2546281
9,Charles,2293389


In [52]:
# Define the query
sql = """
SELECT word, word_count
FROM bigquery-public-data.samples.shakespeare
WHERE corpus = @corpus
AND word_count >= @min_word_count
ORDER BY word_count DESC;
"""

# Define the parameter values in a query job configuration
job_config = bigquery.QueryJobConfig(
    query_parameters= [ 
        bigquery.ScalarQueryParameter("corpus","STRING","romeoandjuliet"),
        bigquery.ScalarQueryParameter("min_word_count","INT64", 250),])

# Start the query job
query_job = client.query(sql, location="US", job_config=job_config)

# Return the resutls as a pandas DataFrame
query_job.to_dataframe()

,word,word_count
0,the,614
1,I,577
2,and,490
3,to,486
4,a,407
5,of,367
6,my,314
7,is,307
8,in,291
9,you,271


In [58]:
# To import rows
project = "pythonproject-330909"
dataset_id = "datascience"

dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table("fraud_dataset_v1")
table = client.get_table(table_ref)

df = client.list_rows(table).to_dataframe()

In [26]:
project = "bigquery-public-data"
dataset_id = "samples"

dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table("shakespeare")
table = client.get_table(table_ref)

df = client.list_rows(table).to_dataframe()

In [8]:
df

,word,word_count,corpus,corpus_date
0,LVII,1,sonnets,0
1,augurs,1,sonnets,0
2,dimm'd,1,sonnets,0
3,plagues,1,sonnets,0
4,treason,1,sonnets,0
...,...,...,...,...
164651,that,232,kinghenryviii,1612
164652,his,234,kinghenryviii,1612
164653,me,235,kinghenryviii,1612
164654,your,242,kinghenryviii,1612


In [56]:
sql = """
    SELECT created_date, complaint_description,
           ST_GEOGPOINT(longitude, latitude) as location
    FROM bigquery-public-data.austin_311.311_service_requests
    LIMIT 10
"""

df = client.query(sql).to_geodataframe()

ValueError: The geopandas library is not installed, please install geopandas to use the to_geodataframe() function.

In [59]:
import datetime

from google.cloud import bigquery
import pandas
import pytz

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of the table to create.
# table_id = "your-project.your_dataset.your_table_name"
table_id = "pythonproject-330909.persondata.table1"
records = [
    {
        "title": u"The Meaning of Life",
        "release_year": 1983,
        "length_minutes": 112.5,
        "release_date": pytz.timezone("Europe/Paris")
        .localize(datetime.datetime(1983, 5, 9, 13, 0, 0))
        .astimezone(pytz.utc),
        # Assume UTC timezone when a datetime object contains no timezone.
        "dvd_release": datetime.datetime(2002, 1, 22, 7, 0, 0),
    },
    {
        "title": u"Monty Python and the Holy Grail",
        "release_year": 1975,
        "length_minutes": 91.5,
        "release_date": pytz.timezone("Europe/London")
        .localize(datetime.datetime(1975, 4, 9, 23, 59, 2))
        .astimezone(pytz.utc),
        "dvd_release": datetime.datetime(2002, 7, 16, 9, 0, 0),
    },
    {
        "title": u"Life of Brian",
        "release_year": 1979,
        "length_minutes": 94.25,
        "release_date": pytz.timezone("America/New_York")
        .localize(datetime.datetime(1979, 8, 17, 23, 59, 5))
        .astimezone(pytz.utc),
        "dvd_release": datetime.datetime(2008, 1, 14, 8, 0, 0),
    },
    {
        "title": u"And Now for Something Completely Different",
        "release_year": 1971,
        "length_minutes": 88.0,
        "release_date": pytz.timezone("Europe/London")
        .localize(datetime.datetime(1971, 9, 28, 23, 59, 7))
        .astimezone(pytz.utc),
        "dvd_release": datetime.datetime(2003, 10, 22, 10, 0, 0),
    },
]
dataframe = pandas.DataFrame(
    records,
    # In the loaded table, the column order reflects the order of the
    # columns in the DataFrame.
    columns=[
        "title",
        "release_year",
        "length_minutes",
        "release_date",
        "dvd_release",
    ],
    # Optionally, set a named index, which can also be written to the
    # BigQuery table.
    index=pandas.Index(
        [u"Q24980", u"Q25043", u"Q24953", u"Q16403"], name="wikidata_id"
    ),
)
job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        bigquery.SchemaField("title", bigquery.enums.SqlTypeNames.STRING),
        # Indexes are written if included in the schema by name.
        bigquery.SchemaField("wikidata_id", bigquery.enums.SqlTypeNames.STRING),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    dataframe, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 4 rows and 6 columns to pythonproject-330909.persondata.table1
